In [6]:
import sys
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import urllib.request
from konlpy.tag import Okt

import json

tokenizer = Tokenizer()

# wordInext 목록 받아오기
with open('LSTM_module_ver2.0_wordIndex.json') as f:
    word_index = json.load(f)
    tokenizer.word_index = word_index
    
with open('LSTM_module_ver2.0_variable.json') as f:
    var = json.load(f)
    
max_data = var['max_data'] # 데이터 문자열 최대 길이
min_data = var['min_data'] # 데이터 문자열 최소 길이
max_len = var['max_len'] # 전체 데이터 셋 길이 설정 (메일의 최대 길이)

model = tf.keras.models.load_model('LSTM_module_ver2.0.h5')
model.summary()

okt = Okt()

def preTreatment(data):
    stopwords = ['\n','.','?',',','']
    temp = okt.morphs(data, stem=True) # 토큰화
    temp = [word for word in temp if not word in stopwords] # 불용어 처리
    return temp


def sentiment_predict(new_sentence):
    data = tokenizer.texts_to_sequences([new_sentence]) # 단어를 숫자값, 인덱스로 변환하여 저장
    print(data)
    pad_new = pad_sequences(data, maxlen = max_len) # 모든 메일의 길이를 100로 설정 (빈 부분은 0으로 패딩)
    score = float(model.predict(pad_new))
    if (score > 0.5):
        print(' {:.2f}% 확률로 스팸입니다.'.format(score * 100))
    else:
        print(' {:.2f}% 확률로 스팸이 아닙니다.'.format((1 - score) * 100))
    
    print('-------------------------------------------------------------------------')
        

# 음성파일 STT 변환
import speech_recognition as sr

r = sr.Recognizer()
harvard = sr.AudioFile('ham2.wav') # 100MB 용량 제한 (약 5분?)
with harvard as source:
    audio = r.record(source)

t = r.recognize_google(audio, language='ko-KR')

temp_X = []
x = [t[i:i+max_data] for i in range(0, len(t), max_data)] # 텍스트 파일 중 150 길이로 데이터 길이 제한

for s in x:
    if len(s) > min_data: # 길이가 50 이하인 데이터 삭제
        temp_X.append(s)
x = temp_X
for txt in x:
    print(txt)
    data = preTreatment(txt)
    sentiment_predict(data)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         2466000   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               117248    
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 2,583,377
Trainable params: 2,583,377
Non-trainable params: 0
_________________________________________________________________
어 재석아 녹화 중 있니 어 예 재석아 나 한 명만 좀 급해 해 줄 수 있니 너 밖에 생각나는 사람이 없다 그래서 좀 주면 내가 내년 내년 말엔 아가파 나는 솔직한 사람이잖아내는 거 같아 지금도 칸이 내일 뭐 누가 있어 어 그 유퀴즈 한번 불러 해 줄게 그러면 이영로 
[[104, 17, 17135, 134, 13, 104, 142, 17, 7, 1, 327, 46, 54, 1617, 136, 353, 160, 13, 38, 439, 705, 71, 2, 25, 109, 54, 106, 19, 4, 1447, 1447, 52, 295, 2546, 1300, 7, 15, 541, 71, 2, 679, 30, 26, 39, 10, 7973, 2, 183, 43, 396